# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
#Drop missing values
spaceship = spaceship.dropna()

# Extract the first letter of the 'Cabin' column
spaceship['Cabin'] = spaceship['Cabin'].str[0]

# Drop unnecessary columns
spaceship = spaceship.drop(columns=['Name', 'PassengerId'])

# Convert boolean columns to integers
spaceship['CryoSleep'] = spaceship['CryoSleep'].astype(int)
spaceship['VIP'] = spaceship['VIP'].astype(int)
spaceship['Transported'] = spaceship['Transported'].astype(int)

# One-hot encoding for categorical variables
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'Destination', 'Cabin'], drop_first=True)

# Convert dummied columns to integers
columns_dummed =[
    'HomePlanet_Europa',
    'HomePlanet_Mars',
    'Destination_PSO J318.5-22',
    'Destination_TRAPPIST-1e',
    'Cabin_B',
    'Cabin_C',
    'Cabin_D',
    'Cabin_E',
    'Cabin_F',
    'Cabin_G',
    'Cabin_T'
    ]
spaceship[columns_dummed] = spaceship[columns_dummed].astype(int)

# Numerical Scaling
from sklearn.preprocessing import MinMaxScaler

# Escalator Min-Max
scaler = MinMaxScaler()
spaceship = pd.DataFrame(
    scaler.fit_transform(spaceship), 
    columns=spaceship.select_dtypes(include=['float64', 'int64']).columns
)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = spaceship.drop('Transported', axis=1)
y = spaceship['Transported']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data: {X_train.shape}")
print(f"Testing data: {X_test.shape}")

model_gb = GradientBoostingClassifier(random_state=42)

model_gb.fit(X_train, y_train)

y_pred = model_gb.predict(X_test)
y_probs = model_gb.predict_proba(X_test)[:, 1]

print(f'Predicciones: {y_pred[:10]}')
print(f'Probabilidades: {y_probs[:10]}')
print(f'Accuracy Score: {accuracy_score(y_test, y_pred):.4f}')

Training data: (5284, 19)
Testing data: (1322, 19)
Predicciones: [1. 1. 1. 1. 0. 0. 1. 0. 0. 1.]
Probabilidades: [0.95588915 0.76593463 0.56842413 0.63780278 0.40456481 0.13942804
 0.92009723 0.04622036 0.12148288 0.92339599]
Accuracy Score: 0.8071


- Evaluate your model

In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión final del modelo: {accuracy:.4f}")

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

Precisión final del modelo: 0.8071

Reporte de Clasificación:
              precision    recall  f1-score   support

         0.0       0.84      0.76      0.79       653
         1.0       0.78      0.86      0.82       669

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [11]:
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0]
}

- Run Grid Search

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

grid_gb = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_grid=param_grid_gb,
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_gb.fit(X_train, y_train)

best_gb = grid_gb.best_estimator_

print(f"Mejores parámetros encontrados: {grid_gb.best_params_}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}


- Evaluate your model

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = best_gb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"--- Evaluación del Modelo ---")
print(f"Accuracy: {accuracy:.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

--- Evaluación del Modelo ---
Accuracy: 0.8139

Matriz de Confusión:
[[496 157]
 [ 89 580]]

Reporte de Clasificación:
              precision    recall  f1-score   support

         0.0       0.85      0.76      0.80       653
         1.0       0.79      0.87      0.83       669

    accuracy                           0.81      1322
   macro avg       0.82      0.81      0.81      1322
weighted avg       0.82      0.81      0.81      1322

